# Limpieza de datos de Alumnos Prioritarios, Preferentes y/o Beneficiarios (SEP)

En este notebook se realizará la limpieza de los datos del esquema de registro de Alumnos Prioritarios, Preferentes y/o Beneficiarios (SEP) de colegios en Chile.

## 1. Importación de librerías
A continuación, se importan las librerías que se utilizarán en el notebook.

In [18]:
import os 
import pandas as pd 

## 2. Carga de datos
Se cargan los datos de los alumnos prioritarios, preferentes y/o beneficiarios.

In [19]:
df = pd.read_csv('../data/SEP.csv', sep=';')

/tmp/ipykernel_11652/1471849577.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/SEP.csv', sep=';')


### 2.1 Columnas a utilizar
Se seleccionan las columnas que se utilizarán en el análisis.


In [20]:
columns = ['MRUN', 'RBD', 'CONVENIO_SEP', 'COD_COM_RBD', 'COD_REG_RBD', 'BEN_SEP', 'CRITERIO_SEP', 'PREFERENTE_ALU', 'PRIORITARIO_ALU']
df = df[columns]

In [21]:
df.head(10)

,MRUN,RBD,CONVENIO_SEP,COD_COM_RBD,COD_REG_RBD,BEN_SEP,CRITERIO_SEP,PREFERENTE_ALU,PRIORITARIO_ALU
0,25,,0,,,0,0,1,0
1,31,10763,1,13604,13,1,1,0,1
2,47,11240,0,5502,5,0,2,0,1
3,48,20364,1,6101,6,1,2,0,1
4,86,3998,1,16107,16,1,1,0,1
5,87,18237,1,8108,8,0,0,1,0
6,88,10506,1,13201,13,1,0,1,0
7,89,2308,0,6115,6,0,0,1,0
8,115,17763,1,8203,8,1,2,0,1
9,116,8492,1,13101,13,1,2,0,1


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999491 entries, 0 to 2999490
Data columns (total 9 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   MRUN             int64 
 1   RBD              object
 2   CONVENIO_SEP     int64 
 3   COD_COM_RBD      object
 4   COD_REG_RBD      object
 5   BEN_SEP          int64 
 6   CRITERIO_SEP     int64 
 7   PREFERENTE_ALU   int64 
 8   PRIORITARIO_ALU  int64 
dtypes: int64(6), object(3)
memory usage: 206.0+ MB


### 2.2 Data Cleaning
A continuación, se realiza la limpieza de los datos.

In [23]:
# Asegura que no existan valores repetidos o nulos
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [24]:
def verificar_valores_atipicos(df, column):
    '''
    Verifica si hay valores atípicos en una columna de un dataframe.

    Parameters:
    - df (DataFrame): dataframe
    - column (str): nombre de la columna

    Returns:
    - resultado (bool): True si hay valores atípicos, False si no los hay.  
    '''
    hay_valor_distinto = (df[column] != 0) & (df[column] != 1)
    resultado = hay_valor_distinto.any()
    return resultado

# Test case: No valores atípicos
assert verificar_valores_atipicos(df, 'BEN_SEP') == False
assert verificar_valores_atipicos(df, 'PREFERENTE_ALU') == False
assert verificar_valores_atipicos(df, 'PRIORITARIO_ALU') == False

print("All tests passed!")


All tests passed!


### 2.3 Cambio de tipo de datos
Se cambia el tipo de datos de las columnas que lo requieran.

In [26]:
# RBD, COD_COM_RBD, COD_REG_RBD deben ser int
# Primero, descartamos las filas con '' o ' '
df = df[(df['RBD'] != '') & (df['RBD'] != ' ')]
df = df[(df['COD_COM_RBD'] != '') & (df['COD_COM_RBD'] != ' ')]
df = df[(df['COD_REG_RBD'] != '') & (df['COD_REG_RBD'] != ' ')]

# Luego, convertimos a int
df['RBD'] = df['RBD'].astype(int)
df['COD_COM_RBD'] = df['COD_COM_RBD'].astype(int)
df['COD_REG_RBD'] = df['COD_REG_RBD'].astype(int)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2668203 entries, 1 to 2994793
Data columns (total 9 columns):
 #   Column           Dtype
---  ------           -----
 0   MRUN             int64
 1   RBD              int64
 2   CONVENIO_SEP     int64
 3   COD_COM_RBD      int64
 4   COD_REG_RBD      int64
 5   BEN_SEP          int64
 6   CRITERIO_SEP     int64
 7   PREFERENTE_ALU   int64
 8   PRIORITARIO_ALU  int64
dtypes: int64(9)
memory usage: 203.6 MB


In [28]:
# En estas lineas trataremos la legibilidad de nuestros datos, modificando nombres de variables 

# Reemplazar los valores en la columna 'PRIORITARIO_ALU'
# df_PPB['PRIORITARIO_ALU'] = df_PPB['PRIORITARIO_ALU'].replace({0: 'No prioritario', 1: 'prioritario'})
# Reemplazar los valores en la columna 'PREFERENTE_ALU'
# df_PPB['PREFERENTE_ALU'] = df_PPB['PREFERENTE_ALU'].replace({0: 'No preferente', 1: 'Preferente'})
# Reemplazar los valores en la columna 'BEN_SEP'
# df_PPB['BEN_SEP'] = df_PPB['BEN_SEP'].replace({0: 'No beneficiario SEP', 1: 'Beneficiario SEP'})

## 3. Filtro de datos
Ahora realizaremos el filtro de los datos para quedarnos con los alumnos prioritarios, preferentes y/o beneficiarios pertenecientes al dataset limpio de 'asistencia.csv'.

In [29]:
df_asistencia = pd.read_csv('output_data/asistencia.csv', sep=',')

In [30]:
# Elimina del dataframe las filas que no pertenecen a los MRUNs de la lista
df = df[df['MRUN'].isin(df_asistencia['RUN_ALUMNO'])]

In [31]:
# Elimina del dataframe las filas que no pertenecen a los RBDs de la lista
df = df[df['RBD'].isin(df_asistencia['RBD'])]

In [32]:
df.head()

,MRUN,RBD,CONVENIO_SEP,COD_COM_RBD,COD_REG_RBD,BEN_SEP,CRITERIO_SEP,PREFERENTE_ALU,PRIORITARIO_ALU
9,116,8492,1,13101,13,1,2,0,1
17,171,9780,1,13105,13,1,0,1,0
35,292,24685,1,13106,13,1,0,1,0
41,360,11818,1,13105,13,1,1,0,1
42,378,25279,1,13119,13,1,0,1,0


## 4. Exportación de datos
Finalmente, se exporta el dataset limpio de alumnos prioritarios, preferentes y/o beneficiarios.

In [33]:
df.to_csv('output_data/SEP.csv', index=False)